## Transform Customer Data
<ol>
<li>Remove records with null customer ids</li>
<li>Remove exact duplicate records</li>
<li>Remove duplicate records based on created_timestamp</li>
<li>Write tranformed data to Silver Schema</li>
</ol>

### 1.Remove records with null customer ids

In [0]:
SELECT * FROM gizmobox_cat.bronze.v_customers where customer_id is not null;

### 2.Remove exact duplicate records

In [0]:
SELECT * FROM gizmobox_cat.bronze.v_customers where customer_id is not null order by customer_id ;

In [0]:
SELECT DISTINCT * FROM gizmobox_cat.bronze.v_customers where customer_id is not null order by customer_id ;

In [0]:
SELECT CUSTOMER_ID,max(created_timestamp),max(customer_name),max(date_of_birth),max(email),
max(member_since),max(telephone )
 FROM gizmobox_cat.bronze.v_customers where customer_id is not null GROUP BY customer_id order by customer_id ;

In [0]:
create or replace temporary view v_customers_distinct as
SELECT DISTINCT * FROM gizmobox_cat.bronze.v_customers where customer_id is not null order by customer_id ;

In [0]:
select * from v_customers_distinct;

### 3.Remove duplicate records based on created_timestamp

In [0]:
select customer_id,max(created_timestamp) as max_created_timestamp from v_customers_distinct group by customer_id

In [0]:
with cte_max as 
(SELECT customer_id,max(created_timestamp) as max_created_timestamp from v_customers_distinct group by customer_id),
cte_max_2 as (
select t.* from v_customers_distinct t join cte_max t1 on t.customer_id = t1.customer_id and t.created_timestamp = t1.max_created_timestamp)
select * from cte_max_2;

### 4.CAST the column values to the correct datatype

In [0]:
with cte_max as 
(SELECT customer_id,max(created_timestamp) as max_created_timestamp from v_customers_distinct group by customer_id),
cte_max_2 as (
select t.* from v_customers_distinct t join cte_max t1 on t.customer_id = t1.customer_id and t.created_timestamp = t1.max_created_timestamp)
select CAST(created_timestamp as timestamp) as created_timestamp,
customer_id,
customer_name,
cast(date_of_birth as DATE) AS DATE_OF_BIRTH,
email,
CAST(member_since as DATE) as member_since,
telephone,
file_path,file_name
 from cte_max_2;

### 5.Write tranformed data to Silver Schema

In [0]:
CREATE TABLE gizmobox_cat.silver.customers as
with cte_max as 
(SELECT customer_id,max(created_timestamp) as max_created_timestamp from v_customers_distinct group by customer_id),
cte_max_2 as (
select t.* from v_customers_distinct t join cte_max t1 on t.customer_id = t1.customer_id and t.created_timestamp = t1.max_created_timestamp)
select CAST(created_timestamp as timestamp) as created_timestamp,
customer_id,
customer_name,
cast(date_of_birth as DATE) AS DATE_OF_BIRTH,
email,
CAST(member_since as DATE) as member_since,
telephone,
file_path,file_name
 from cte_max_2;

In [0]:
select * from gizmobox_cat.silver.customers;

In [0]:
DESCRIBE EXTENDED gizmobox_cat.silver.customers;